In [3]:
# !pip install -q selenium==4.3.0
# !pip install lxml
# !pip install -q beautifulsoup4==4.11.1
# !pip install -q backoff==2.1.2
# import backoff
# backoff.__version__
# !pip install -q pandas
# !pip install python-dotenv

In [4]:
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys # 鍵盤事件
import pandas as pd
import backoff
from datetime import datetime

In [5]:
def set_ua():
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
    return user_agent

In [6]:
@backoff.on_exception(backoff.expo,
                        Exception, 
                      max_time=10)
def chrome_init():
    chrome_opt = webdriver.ChromeOptions()
    # chrome_opt.add_argument('--headless')
    chrome_opt.add_argument('--no-sandbox')
    chrome_opt.add_argument('--ignore-ssl-errors=yes')
    chrome_opt.add_argument('--ignore-certificate-errors')
    chrome_opt.add_argument(f'user-agent={set_ua()}')
    # chrome_opt.add_argument("--incognito")  # 使用無痕模式。用 selenium開瀏覽器已經很乾淨了，但疑心病重的可以用一下
    driver = webdriver.Remote(
        command_executor='http://selenium-hub:4444/wd/hub',
        options=chrome_opt
    )
    
    return driver

In [7]:
url = 'https://store.steampowered.com/category/action/' # 指定網址
driver = chrome_init()
driver.get(url) # 進入指定網址
driver.implicitly_wait(5) # 等待伺服器反應最多 5 秒，如果在時間到之前反應就提早結束等待

In [8]:
n = 0

while n < 3:
    try:
        butt = driver.find_element('css selector', '#SaleSection_13268 > div._1cOoCFwafBlSkwllIMf3XM._39HWXhhjsbML7K9sme9ItV.SaleSectionForCustomCSS > div._18byEIHFiivSklOwKqIx2b > div:nth-child(2) > div._3vjDu6zylspBzUE7FmM6Yl > div > div._36qA-3ePJIusV1oKLQep-w > button')
        if butt:
            butt.click()
            n += 1
            print(n)
            sleep(2)
            
    except:
        driver.find_element('css selector', 'html').send_keys(Keys.PAGE_DOWN) # 通常定位在最根部的 html 即可
        sleep(0.5)

1
2
3


In [9]:


# 獲取頁面源代碼
html_source = driver.page_source

# 使用 BeautifulSoup 解析 HTML 內容
soup = BeautifulSoup(html_source, 'lxml')

all = soup.find_all("div", class_ = "v9uRg57bwOaPsvAnkXESO")

columns = ['遊戲名稱', 'APPID', '發行日期', '評價', '使用者評論數', '價格', '遊戲標籤']

save_lst = []
for rank in all:    

    appid_href = rank.find_all('div', class_='_3rrH9dPdtHVRMzAEw82AId')

    # get app id
    for appids in appid_href:
        appid = appids.find('a', href=True)
        if appid:
            href = appid['href']
            app_id = href.split('/')[4]


    tags = rank.find_all('a', class_='_33yqka47vWurNqhnhLJb_m')
    name = rank.find('div', class_='_2ekpT6PjwtcFaT4jLQehUK StoreSaleWidgetTitle').text
    date = rank.find('div', class_='_1qvTFgmehUzbdYM9cw0eS7').text


    rating = rank.find('div', class_='_3ZWs0kB-1tuqQtie9KK-E7')
    if rating:
        comment = rating.find('div').text
        num_comment = rating.find('div', class_='_1wXL_MfRpdKQ3wZiNP5lrH').text
    else:
        comment = ""
        num_comment = ""
        
    costs = rank.find_all('div', class_='_3j4dI1yA7cRfCvK8h406OB')    
    for cost in costs:                                
        cost = cost.text

    value = []
    for tag in tags:                                
        value.append(tag.text)

        result = [name, app_id, date, comment, num_comment, cost, value]
    
    save_lst.append(result)


df = pd.DataFrame(
    data = save_lst,
    columns = columns
)


driver.quit()

In [10]:
df

df.to_csv('steam_review_info.csv', index = 0, encoding = "utf-8")